# Trabajo Practico KEY

### Importo las librerias necesarias

In [1]:
import pandas as pd
import numpy as np
import dtale as dt
import plotly.express as px
from sklearn import neighbors
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer  
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.svm import LinearSVC

### Importo los Data Frames

In [2]:
tc_mano = pd.read_csv('tiempos_cambio_mano.csv',sep=";")
tp_tecla = pd.read_csv('tiempos_por_tecla.csv',sep=";")
usuarios = pd.read_csv('usuarios.csv',sep=";")

### Ordeno y genero una columna con el valor de la tecla en ASCII (no es necesario pero es mas comodo)

In [3]:
tp_tecla = tp_tecla[["tiempo_id","usuario","practica","tecla","down","up","anterior","anterior_down","modo"]]
#tp_tecla["ant_up"] = tp_tecla.up.shift(periods=1)

### Genero una copia del df agrupando por usuario/practica tomando 31 caracteres y elijiendo el modo de escritura "corto" a la vez que elimino columnas inecesarias.

In [4]:
tp_tecla1 = tp_tecla[(tp_tecla.modo == 1)].copy()
tp_tecla1 = tp_tecla1.groupby(["usuario","practica"]).head(33).reset_index(drop=True)
tp_tecla1 = tp_tecla1.drop(['anterior',"modo"], axis=1)
tp_tecla1["ant_up"] = tp_tecla1.up.shift(periods=1)
tp_tecla1["ant_up"] = np.where(tp_tecla1["down"] == 0 , 0 , tp_tecla1["ant_up"])


In [5]:
a = []
for x in tp_tecla1["tecla"]:
    a.append(chr(x))
tp_tecla1['ascii'] = a
tp_tecla1=tp_tecla1[(tp_tecla1.ascii != ".") & (tp_tecla1.ascii != ",")].copy()
tp_tecla1['pp'] = tp_tecla1['ascii'].str.extract('(\D)')
tp_tecla1 = tp_tecla1.dropna(subset=["pp"])
tp_tecla1 = tp_tecla1.drop("pp",axis=1)
df_key = tp_tecla1.copy()
tp_tecla1#.head(32)

,tiempo_id,usuario,practica,tecla,down,up,anterior_down,ant_up,ascii
0,1,1,1,69,0,43,0,0.0,E
1,2,1,1,83,157,237,0,43.0,S
2,3,1,1,84,330,458,157,237.0,T
3,4,1,1,65,426,571,330,458.0,A
4,5,1,1,32,572,650,426,571.0,
...,...,...,...,...,...,...,...,...,...
18210,239244,98,2,84,4904,4983,4619,4684.0,T
18211,239245,98,2,85,5115,5183,4904,4983.0,U
18212,239246,98,2,82,5258,5323,5115,5183.0,R
18213,239247,98,2,65,5463,5588,5258,5323.0,A


### Genero nuevas columnas con diferentes tiempo de pulsado entre teclas

In [6]:
df_key["hold_time"] = df_key['up'] - df_key['down']
df_key["log_relase_time"] = np.log(df_key['up'] - df_key['anterior_down'])
df_key["down-ant_down"] = df_key['down'] - df_key['anterior_down']
df_key["down-ant_up"] = df_key['down'] - df_key['ant_up']
df_key["up-ant_up"] = df_key['up'] - df_key['ant_up']
df_key.head(31)

,tiempo_id,usuario,practica,tecla,down,up,anterior_down,ant_up,ascii,hold_time,log_relase_time,down-ant_down,down-ant_up,up-ant_up
0,1,1,1,69,0,43,0,0.0,E,43,3.761200,0,0.0,43.0
1,2,1,1,83,157,237,0,43.0,S,80,5.468060,157,114.0,194.0
2,3,1,1,84,330,458,157,237.0,T,128,5.707110,173,93.0,221.0
3,4,1,1,65,426,571,330,458.0,A,145,5.484797,96,-32.0,113.0
4,5,1,1,32,572,650,426,571.0,,78,5.411646,146,1.0,79.0
5,6,1,1,69,714,778,572,650.0,E,64,5.327876,142,64.0,128.0
6,7,1,1,83,858,970,714,778.0,S,112,5.545177,144,80.0,192.0
7,8,1,1,32,986,1066,858,970.0,,80,5.337538,128,16.0,96.0
8,9,1,1,85,1146,1226,986,1066.0,U,80,5.480639,160,80.0,160.0
9,10,1,1,78,1338,1418,1146,1226.0,N,80,5.605802,192,112.0,192.0


<img src="key.jpeg">

In [9]:
ej = pd.pivot_table(df_key, values=["hold_time","log_relase_time","down-ant_down","down-ant_up","up-ant_up"],
                    index=["usuario","practica"],columns=["ascii"])
ej

down-ant_down                                              \
ascii                                A       B      C      D      E      I   
usuario practica                                                             
1       1           130.333333   80.50   192.0  193.0  128.0  111.6   81.0   
        2           240.166667   83.25   192.0  207.0  159.0  125.0   97.0   
        3           151.666667  179.75   191.0  205.0  143.0  131.4   66.0   
        4           123.333333   91.25   161.0  224.0  127.0  136.8  126.0   
        5           130.166667   83.00   194.0  175.0  144.0  127.8   80.0   
...                        ...     ...     ...    ...    ...    ...    ...   
102     1           252.666667  216.25   250.0  294.0  200.0  212.4  333.0   
        2           215.500000  240.00   440.0  313.0  725.0  158.6  426.0   
        3           285.000000  251.00  1214.0  314.0  448.0  224.0  268.0   
        4           279.500000  290.00   264.0  284.0  586.0  191.0  461.0   
        5           207.166667  276.75   279.0  318.0  192.0  255.0  277.0   

                                            ... up-ant_up                \
ascii                 N      P           R  ...         C      D      E   
usuario practica                            ...                           
1       1         192.0  159.0  180.666667  ...     192.0  143.0  107.8   
        2         272.0  128.0  165.666667  ...     192.0  144.0  115.0   
        3         624.0  240.0  170.666667  ...     189.0  145.0  140.6   
        4         192.0  113.0  176.333333  ...     208.0  144.0  140.8   
        5         176.0  160.0  202.333333  ...     192.0  143.0  134.0   
...                 ...    ...         ...  ...       ...    ...    ...   
102     1         248.0  215.0  318.000000  ...     326.0  160.0  205.8   
        2         232.0  394.0  216.000000  ...     292.0  713.0  160.2   
        3         226.0  329.0  249.666667  ...     327.0  449.0  221.2   
        4         279.0  246.0  258.666667  ...     303.0  573.0  186.6   
        5         225.0  332.0  269.666667  ...     327.0  163.0  273.8   

                                                                      \
ascii                 I      N      P           R           S      T   
usuario practica                                                       
1       1          79.0  192.0  224.0  186.333333  187.666667  192.0   
        2          79.0  255.0  144.0  171.333333  197.333333  176.0   
        3          95.0  625.0  240.0  166.000000  194.333333  183.0   
        4         143.0  223.0  111.0  177.000000  197.666667  209.0   
        5          80.0  192.0  129.0  181.333333  197.333333  169.5   
...                 ...    ...    ...         ...         ...    ...   
102     1         308.0  257.0  199.0  314.333333  266.000000  261.0   
        2         383.0  246.0  418.0  213.000000  316.000000  216.5   
        3         244.0  243.0  325.0  234.666667  343.666667  190.0   
        4         466.0  283.0  198.0  244.666667  241.666667  472.0   
        5         247.0  238.0  336.0  257.333333  236.666667  265.0   

                               
ascii                       U  
usuario practica               
1       1          133.000000  
        2          432.000000  
        3          133.000000  
        4           95.666667  
        5           89.333333  
...                       ...  
102     1          310.333333  
        2          356.000000  
        3         1221.000000  
        4          419.666667  
        5          236.666667  

[552 rows x 65 columns]

In [9]:
X = df_key.drop(["tiempo_id","usuario","up","practica","down","anterior_down","ant_up","ascii"],axis=1)
Y = df_key["usuario"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 100)
X_val_train, X_val_test, Y_val_train, Y_val_test = train_test_split(X_train, Y_train, test_size=0.2, random_state = 100)

In [50]:
from sklearn.preprocessing import StandardScaler
Standard = StandardScaler()
X_val_train_Standard = Standard.fit_transform(X_val_train)
X_val_test_Standard = Standard.transform(X_val_test)

In [51]:
from sklearn import neighbors
from sklearn.metrics import mean_squared_error, r2_score
knn_Standard = neighbors.KNeighborsClassifier(n_neighbors=10,weights='distance',p=2)
knn_Standard.fit(X_val_train_Standard,Y_val_train) 
knn_pred_Standard = knn_Standard.predict(X_val_test_Standard)

knn_Standard1 = mean_squared_error(Y_val_test, knn_pred_Standard ,squared=False)
print("Evaluación modelo kNN con Standard")
print("RMSE 12 vecinos:",round(knn_Standard1,3))

Evaluación modelo kNN con Standard
RMSE 12 vecinos: 20.963


In [52]:
from sklearn.tree import DecisionTreeClassifier
modelo_arbol_Standard = DecisionTreeClassifier(max_depth=5, random_state=100)       
modelo_arbol_Standard.fit(X_val_train_Standard, Y_val_train)              
arbol_pre_Standard = modelo_arbol_Standard.predict(X_val_test_Standard)


RMSE_tree_Standard=mean_squared_error(Y_val_test, arbol_pre_Standard,squared=False)
print("Evaluación del modelo Seleccionado: ")
print("RMSE con Standard y 6 ramas:",round(RMSE_tree_Standard,3))

Evaluación del modelo Seleccionado: 
RMSE con Standard y 6 ramas: 20.273


In [ ]:
from sklearn.linear_model import ElasticNet
enet_Standard = ElasticNet(alpha=0.9, l1_ratio=0.1)
enet_Standard.fit(X_val_train_Standard, Y_val_train)
y_pred_enet_Standard = enet_Standard.predict(X_val_test_Standard)

RMSE_enet_Standard=mean_squared_error(Y_val_test, y_pred_enet_Standard,squared=False)
print("RMSE_Ridge: ",round(RMSE_enet_Standard,3))

from sklearn.metrics import f1_score, accuracy_score
"""F1=f1_score(Y_val_test,y_pred_enet_Standard, average='macro')
print("F1: ", round(F1,3))"""


AC=accuracy_score(Y_val_test, y_pred_enet_Standard)
print("Accuracy: ", round(AC,3))

In [12]:
"""from sklearn.metrics import mean_squared_error, r2_score
reg_log = LogisticRegression(max_iter=5000,fit_intercept=True,random_state=100)
reg_log.fit(X = X_val_train, y = Y_val_train)
y_predict = reg_log.predict(X_val_test)
accuracy_reg = accuracy_score(y_true = Y_val_test, y_predict = y_predict)
accuracy_reg
knn_prul1=mean_squared_error(Y_val_test, y_predict,squared=False)
print("Evaluación ")
print("RMSE:",round(knn_prul1,3))"""

'from sklearn.metrics import mean_squared_error, r2_score\nreg_log = LogisticRegression(max_iter=5000,fit_intercept=True,random_state=100)\nreg_log.fit(X = X_val_train, y = Y_val_train)\ny_predict = reg_log.predict(X_val_test)\naccuracy_reg = accuracy_score(y_true = Y_val_test, y_predict = y_predict)\naccuracy_reg\nknn_prul1=mean_squared_error(Y_val_test, y_predict,squared=False)\nprint("Evaluación ")\nprint("RMSE:",round(knn_prul1,3))'

In [9]:
# tratabas de hacer el modelo pero te diste cuenta que habia que hacer mas variables

In [10]:
"""posible_drop = matcopy2[(matcopy2["down-ant_down"]>=2000)]
posible_drop"""

'posible_drop = matcopy2[(matcopy2["down-ant_down"]>=2000)]\nposible_drop'

In [11]:
"""
h=df_key[df_key['tecla']==32][["usuario",'down','up']]
h["ant_up"] = h.up.shift(periods=1)
h.loc[h.ant_up.isnull() , 'ant_up'] = 0
h["tiempo_palabra"]= h['down'] - h['ant_up']
#h = h.drop("ant_up",axis=1)
h
"""
#no lo hago porque sino generalizo y no puedo evitar el error general. clase 14/9/21

'\nh=df_key[df_key[\'tecla\']==32][["usuario",\'down\',\'up\']]\nh["ant_up"] = h.up.shift(periods=1)\nh.loc[h.ant_up.isnull() , \'ant_up\'] = 0\nh["tiempo_palabra"]= h[\'down\'] - h[\'ant_up\']\n#h = h.drop("ant_up",axis=1)\nh\n'